<a href="https://colab.research.google.com/github/skytmddus27/president_election/blob/master/%ED%8A%B8%EC%9C%84%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%AC_me.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# jupyter notebook 내 그래프를 바로 그리기 위한 설정
%matplotlib inline

# unicode minus를 사용하지 않기 위한 설정 (minus 깨짐현상 방지)
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'AppleGothic'

In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 15.3 MB/s 
     |████████████████████████████████| 6.6 MB 44.4 MB/s 
     |████████████████████████████████| 77 kB 6.8 MB/s 
     |████████████████████████████████| 596 kB 56.9 MB/s 
     |████████████████████████████████| 895 kB 52.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import EarlyStoppingCallback
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import string

import nltk
from nltk.corpus import stopwords

In [ ]:
if torch.cuda.is_available():    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
park = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kaggle/president election/data/park.csv')
fire = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kaggle/president election/data/fire.csv')
ahn = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kaggle/president election/data/ahn.csv')
number = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kaggle/president election/data/number.csv')

In [ ]:
import re
def clean_tweets(text):
  text = re.sub("RT @[\w]*:","",str(text))
  text = re.sub("@[\w]*","",str(text))
  text = re.sub("https?://[A-Za-z0-9./]*","",str(text))
  text = re.sub("\n","",str(text))
  return str(text)

In [ ]:
park['Text'] = park['Text'].apply(lambda x: clean_tweets(x))
fire['Text'] = fire['Text'].apply(lambda x: clean_tweets(x))
ahn['Text'] = ahn['Text'].apply(lambda x: clean_tweets(x))
number['Text'] = number['Text'].apply(lambda x: clean_tweets(x))

In [ ]:
ahn = ahn.dropna()

In [ ]:
model_name = "beomi/kcbert-base"
tokenizer = BertTokenizer.from_pretrained(model_name)
model_path = "/content/drive/MyDrive/Colab Notebooks/kaggle/president election/data/checkpoint-1500"
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

Downloading:   0%|          | 0.00/244k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

In [ ]:
def convert_input_data(sentences):

    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    MAX_LEN = 64

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
        
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [ ]:
def test_sentences(sentences):
 
    # 평가모드로 변경!!!!!
    model.eval()

    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [ ]:
def preprocessing(df):

    df.document=df.Text.replace('[^A-Za-zㄱ-ㅎㅏ-ㅣ가-힣]+','')

    return df

number = preprocessing(number)
# print(ahn)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
sens =[]
for sen in number.Text:
    sens.append(sen)
# print(len(sens), len(ahn)) # 개수 확인 
# print(sens[:10])

In [ ]:
def make_predicted_label(sen):
    sen = [sen]
    score = test_sentences(sen)
    result = np.argmax(score)

    if result == 0:   # negative 
        return 0
    else:
        return 1   # positive

In [ ]:
model.to(device)

In [ ]:
scores_data=[]
for sen in sens:
    scores_data.append(make_predicted_label(sen))

In [ ]:
number['pred'] = scores_data
number[:50]

,Unnamed: 0,Datetime,Tweet Id,Text,pred
0,0,2022-03-09 07:42:23+00:00,1501463391508250626,Dragonsky Collaboration #4번 작품명 : 우주 최강 빌런 곤드 ...,1
1,1,2022-03-09 05:06:16+00:00,1501424107334021120,1번 G-G#-A-A#-B-C-C#-D-D#-E-F-F#-G 2번 D-D#-E-F-...,0
2,2,2022-03-09 05:03:46+00:00,1501423476120293377,#국민의당 #안철수#4번 #권영현4번 권영현 후보님을대구 중남구 국회의원으로꼭 선택...,1
3,3,2022-03-08 06:06:17+00:00,1501076822221475843,1번 G-G#-A-A#-B-C-C#-D-D#-E-F-F#-G 2번 D-D#-E-F-...,0
4,4,2022-03-07 10:42:03+00:00,1500783832898756611,"님들아 들어봐 4번이 진짜 맛도리야야구방망이에 못 박혀있고때마침 그녀, 의인화가 ...",1
5,5,2022-03-07 07:06:16+00:00,1500729527139389446,1번 G-G#-A-A#-B-C-C#-D-D#-E-F-F#-G 2번 D-D#-E-F-...,0
6,6,2022-03-06 13:34:47+00:00,1500464912963084298,#국민의당 #안철수#4번 #권영현4번 권영현 후보님을대구 중남구 국회의원으로꼭 선택...,1
7,7,2022-03-06 05:06:16+00:00,1500336940784435204,1번 G-G#-A-A#-B-C-C#-D-D#-E-F-F#-G 2번 D-D#-E-F-...,0
8,8,2022-03-05 13:06:17+00:00,1500095352346910721,1번 G-G#-A-A#-B-C-C#-D-D#-E-F-F#-G 2번 D-D#-E-F-...,0
9,9,2022-03-05 09:46:26+00:00,1500045060632887297,지금 윤석열후보와 안철수 전 후보님께서 유세하시는 뉴스 봤는데 눈물 나네요. 안철...,1


In [ ]:
# ahn[ahn['Unnamed: 0'] == '7923']

,Unnamed: 0,Datetime,Tweet Id,Text,pred
7925,7923,2022-03-05 11:08:19+00:00,1.500066e+18,맘 정했음난 안철수(윤석열)에게 투표하기로함그가 진행하는 정치의길에 힘을 실어주기로...,1
22934,7923,2022-03-01 07:37:43+00:00,1.498563e+18,"인명진 등 安 지지 4명 ""단일화 결렬..안철수 지지 철회"" | 다음 뉴스 갈사람...",1
37954,7923,2022-02-21 04:40:15+00:00,1.495619e+18,안빠들! 바보 등신같이 이거 나르지마안철수 최저 득표율 5.8%,1
52470,7923,2022-02-13 00:32:36+00:00,1.492658e+18,"[단독] 안철수, 오늘 대통령 후보 등록 뒤 야권 단일화 논의 제안국민의당 최진석 ...",1


In [ ]:
number.to_csv('/content/drive/MyDrive/Colab Notebooks/kaggle/president election/data/number_pred.csv', index = False)